Testing

# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission, if necessary. Sections that begin with **'Implementation'** in the header indicate where you should begin your implementation for your project. Note that some sections of implementation are optional, and will be marked with **'Optional'** in the header.

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---

## Step 1: Dataset Exploration

Visualize the German Traffic Signs Dataset. This is open ended, some suggestions include: plotting traffic signs images, plotting the count of each sign, etc. Be creative!


The pickled data is a dictionary with 4 key/value pairs:

- features -> the images pixel values, (width, height, channels)
- labels -> the label of the traffic sign
- sizes -> the original width and height of the image, (width, height)
- coords -> coordinates of a bounding box around the sign in the image, (x1, y1, x2, y2). Based the original image (not the resized version).

In [1]:
import pickle
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plot
#import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
#import cv2
import hashlib
import os
from urllib.request import urlretrieve
from PIL import Image
from sklearn.utils import resample
from tqdm import tqdm
from zipfile import ZipFile

from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math


In [2]:
# Load pickled data

# TODO: fill this in based on where you saved the training and testing data
training_file = "train.p"
testing_file = "test.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
    #print(train)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    #print (test)
    
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [3]:
### To start off let's do a basic data summary.

# TODO: number of training examples
n_train = len(X_train)

# TODO: number of testing examples
n_test = len(X_test)

# TODO: what's the shape of an image?
images = X_train[0]
images_shape = np.shape(images)

# TODO: how many classes are in the dataset
n_classes = len(np.unique(y_train))

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", images_shape)
print("Number of classes =", n_classes)



Number of training examples = 39209
Number of testing examples = 12630
Image data shape = (32, 32, 3)
Number of classes = 43


In [4]:
### Data exploration visualization goes here.
### Feel free to use as many code cells as needed.

In [5]:
#ax = sns.countplot(x="labels", data=train)

In [6]:
def show_images(images):
        
    #Split the figure in 3 rows and 3 columns to put images
    figure, axes = plot.subplots(10,10)
    
    # hspace vertical space reserve for white space between subplots
    # wspace horizontal space reserve for white space between subplots
    figure.subplots_adjust(hspace=0.2, wspace=0.2)
    
    for i, ax in enumerate(axes.flat):
        image_shape = np.shape(images[i])
        ax.imshow(images[i].reshape(image_shape), cmap='binary')
        #xlabel = i 

        #Show the number of image as the label on the x-axis.
        #ax.set_xlabel(xlabel)
        
        ax.set_xticks([])
        ax.set_yticks([])
        
    plot.show()

In [7]:
#sample_train_images = X_train[0:100] 
#show_images(sample_train_images)

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

There are various aspects to consider when thinking about this problem:

- Your model can be derived from a deep feedforward net or a deep convolutional network.
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [8]:
### Preprocess the data here.
### Feel free to use as many code cells as needed.

In [9]:
#def grayscale(array):
#    """Applies the Grayscale transform
#    This will return an image with only one color channel
#    but NOTE: to see the returned image as grayscale
#    you should call plt.imshow(gray, cmap='gray')"""
#    greyscale_data = []
#
#    for i in range(0,array.shape[0]):
#        image = array[i]
#        grey_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
#        greyscale_data.append(grey_image)
#        
#    return np.array(greyscale_data)


def grayscale(array):
    greyscale_data = []

    for i in range(0,array.shape[0]):
        image = array[i]        
        grey_image = 0.2989 * image[:,:,0] + 0.5870 * image[:,:,1] + 0.1140 * image[:,:,2]
        greyscale_data.append(grey_image)
        
        
    return np.array(greyscale_data)

X_train = grayscale(X_train)
X_test = grayscale(X_test)

print (X_train.shape)
print (X_test.shape)


(39209, 32, 32)
(12630, 32, 32)


In [10]:
def flatten_gray(array):
    flat_data = []
    
    for i in range(0,array.shape[0]):
        image = array[i]
        #Return a copy of the array collapsed into one dimension.
        data = np.array (image, dtype = np.float32).flatten()
        flat_data.append(data)
        
    return np.array(flat_data)

X_train = flatten_gray(X_train)
X_test = flatten_gray(X_test)

print (X_train.shape)
print (X_test.shape)

(39209, 1024)
(12630, 1024)


In [11]:
def normalize_greyscale(image_data):
    """
    Normalize the image data with Min-Max scaling to a range of [0.1, 0.9]
    :param image_data: The image data to be normalized
    :return: Normalized image data
    """
    # ToDo: Implement Min-Max scaling for greyscale image data
    #costants
    a = 0.1
    b = 0.9
    xmin = 0
    xmax = 255
    
    x = image_data
    
    x_prime = a + ((x-xmin)*(b-a))/(xmax-xmin)
    
    return x_prime



X_train = normalize_greyscale(X_train)
X_test = normalize_greyscale(X_test)

print (X_train.shape)
print (X_test.shape)

(39209, 1024)
(12630, 1024)


In [12]:
# Turn labels into numbers and apply One-Hot Encoding
encoder = LabelBinarizer()
encoder.fit(y_train)
#clases = encoder.classes_
#print (clases)
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

# Change to float32, so it can be multiplied against the features in TensorFlow, which are float32

y_train = y_train.astype(np.float32)

y_test = y_test.astype(np.float32)
print (X_train.shape)


(39209, 1024)


In [13]:
# Get randomized datasets for training and validation

#test_size : represent the proportion of the dataset to include in the test split. 
#If int, represents the absolute number of test samples

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train,test_size=0.01,random_state=832289)

print (X_train.shape)
print (X_valid.shape)
print (y_train.shape)
print (y_valid.shape)
print('Training features and labels randomized and split.')

(38816, 1024)
(393, 1024)
(38816, 43)
(393, 43)
Training features and labels randomized and split.


In [14]:
### Define your architecture here.
### Feel free to use as many code cells as needed.

In [15]:
import tensorflow as tf
sess = tf.InteractiveSession()

In [16]:
#creating nodes for the input images and target output classes.
x = tf.placeholder(tf.float32, shape=[None, 1024])
y_ = tf.placeholder(tf.float32, shape=[None, 43])

In [17]:
#define the weights W and biases b for model
W = tf.Variable(tf.zeros([1024,43]))
b = tf.Variable(tf.zeros([43]))

In [18]:
#Create random weights with zero mean and standard deviation sigma 
#Select a small standard deviation, so that the distributions is very uncertain and improve over time

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    
    return tf.Variable(initial)

#Create bias initialized in zero.
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    
    return tf.Variable(initial)

In [19]:
#Function for Convolution
def conv2d(x, W):
    
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

In [20]:
#Function for MAX-pooling
def max_pool_2x2(x):
    
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [21]:
#First Convolutional Layer
#The convolutional will compute 32 features for each 5x5 patch
W_conv1 = weight_variable([5, 5, 1, 32])

#The bias vector has a component for each output channel.
b_conv1 = bias_variable([32])

In [22]:
#Image shape 32x32, and 1 channel because the image is in Grayscale
#Resahep x to a 4d tensor
x_image = tf.reshape(x, [-1,32,32,1])
#print(tf.shape(x_image))

In [23]:
#Apply a one liner to convolve the x_image with weight tensor and add the bias. Apply the ReLu function.
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

#finally Max pooling
h_pool1 = max_pool_2x2(h_conv1)

In [24]:
#Second Convolutional Layer
#64 features for each 5x5 patch.
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [25]:
#Fully Connected Layer with 1024 neurons
W_fc1 = weight_variable([8 * 8 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 8*8*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [26]:
#Dropout, in order to reduce overfitting.

#placeholder of the probability that the neuron is kept during dropout
keep_prob = tf.placeholder(tf.float32)

h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [27]:
#Readout layer

W_fc2 = weight_variable([1024, 43])
b_fc2 = bias_variable([43])

#Logits
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2  


In [28]:
#Average cross-entropy --> Loss 
# tf.nn.softmax_cross_entropy_with_logits(logits, labels, dim=-1, name=None)
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_conv, y_))

#learning_rate = 9e-4
## This is the method used to train the model
train_step = tf.train.AdamOptimizer(8.8e-4).minimize(cross_entropy)
#optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
 
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))




# Train and Evaluate the model

In [29]:
import datetime


batch_size = 100
total_samples = X_train.shape[0]
print (total_samples)
epochs = 1
n = 10
batch_index = 0
#Loops (at batch size) to get through data once.
loops = int(total_samples / batch_size) 
print (loops)
sess.run(tf.initialize_all_variables())

38816
388


In [30]:
def next_batch(x, y, batch_index, batch_size):
    start = batch_index
    end = start + batch_size
    print(start, end)

    return x[start:end], y[start:end]

In [31]:
#returns accuracy of model
def test_accuracy(target,predictions):
    return(100.0*np.sum(np.argmax(target,1) == np.argmax(predictions,1))/target.shape[0])


In [33]:
import datetime
test_batch_size = 100

print("Start ",datetime.datetime.now().time())

for i in range(epochs):
    for j in range(loops):
        batch_start = j*batch_size
        batch_features = X_train[batch_start:(batch_start + batch_size)]
        batch_labels = y_train[batch_start:(batch_start + batch_size)] 

        if j%100 == 0:
            train_accuracy = accuracy.eval(feed_dict={x:batch_features, y_: batch_labels, keep_prob: 1.0})
            print("step %d, training accuracy %g, batch start %d"%(j, train_accuracy, batch_start))

        train_step.run(feed_dict={x: batch_features, y_: batch_labels, keep_prob: 0.5})
    print("valid accuracy %g"%accuracy.eval(feed_dict={x: X_valid, y_: y_valid, keep_prob: 1.0}))
    
    
    if j == (loops-1):
        result = np.array([]).reshape(0,10)
        loops_test = int(X_test.shape[0]/test_batch_size)
        for i in range (loops_test):
            X_test_batch, y_test_batch = next_batch(X_test, y_test, i, test_batch_size)
            preds = accuracy.eval(feed_dict={x:X_test_batch, y_: y_test_batch, keep_prob: 1.0})
            
            result=np.concatenate((result,preds),axis=0)
            print(len(result))
         
        print ("test accuracy: "+ str(test_accuracy(X_test,result)))
        
print ("Finished ", datetime.datetime.now().time())

Start  06:59:54.936863
step 0, training accuracy 0.86, batch start 0
step 100, training accuracy 0.92, batch start 10000
step 200, training accuracy 0.97, batch start 20000
step 300, training accuracy 0.93, batch start 30000
valid accuracy 0.961832
0 100


ValueError: all the input arrays must have same number of dimensions

### Question 1 

_Describe the techniques used to preprocess the data._

**Answer:**

Convert from RGB to grayscale, since the data doesn't depend on the colors on the image is easier to work with just 1 channel.
Flatten the data into a big numpy array.
Normalize the data in order to have data with zero mean and equal variance.

In [ ]:
### Generate data additional (if you want to!)
### and split the data into training/validation/testing sets here.
### Feel free to use as many code cells as needed.

### Question 2

_Describe how you set up the training, validation and testing data for your model. If you generated additional data, why?_

**Answer:**

I use the function train_test_split from sklearn, which splits the input arrays into random train and test subsets.

### Question 3

_What does your final architecture look like? (Type of model, layers, sizes, connectivity, etc.)  For reference on how to build a deep neural network using TensorFlow, see [Deep Neural Network in TensorFlow
](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/b516a270-8600-4f93-a0a3-20dfeabe5da6/concepts/83a3a2a2-a9bd-4b7b-95b0-eb924ab14432) from the classroom._


I used a Convnet with 2 layers with a convolution on the middle
sizes:
Connectivity:


**Answer:**

In [ ]:
### Train your model here.
### Feel free to use as many code cells as needed.

### Question 4

_How did you train your model? (Type of optimizer, batch size, epochs, hyperparameters, etc.)_


**Answer:**

Optimizer:
Batch size:
epochs:
Hyperparameters:


### Question 5


_What approach did you take in coming up with a solution to this problem?_

**Answer:**

---

## Step 3: Test a Model on New Images

Take several pictures of traffic signs that you find on the web or around you (at least five), and run them through your classifier on your computer to produce example results. The classifier might not recognize some local signs but it could prove interesting nonetheless.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

### Question 6

_Choose five candidate images of traffic signs and provide them in the report. Are there any particular qualities of the image(s) that might make classification difficult? It would be helpful to plot the images in the notebook._



**Answer:**

In [ ]:
### Run the predictions here.
### Feel free to use as many code cells as needed.

### Question 7

_Is your model able to perform equally well on captured pictures or a live camera stream when compared to testing on the dataset?_


**Answer:**

In [ ]:
### Visualize the softmax probabilities here.
### Feel free to use as many code cells as needed.

### Question 8

*Use the model's softmax probabilities to visualize the **certainty** of its predictions, [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.11/api_docs/python/nn.html#top_k) could prove helpful here. Which predictions is the model certain of? Uncertain? If the model was incorrect in its initial prediction, does the correct prediction appear in the top k? (k should be 5 at most)*


**Answer:**

### Question 9
_If necessary, provide documentation for how an interface was built for your model to load and classify newly-acquired images._


**Answer:**

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.